In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


BRISK - CANNY

In [ ]:
import os
import cv2
import numpy as np
from skimage import io
from skimage.metrics import structural_similarity as ssim, mean_squared_error as mse
import matplotlib.pyplot as plt
from multiprocessing import Pool

# Load image with OpenCV
def load_image_cv(path):
    return cv2.imread(path, cv2.IMREAD_COLOR)

def process_image_pair(args):
    filename_a, filename_b, filename_label = args
    img1_path = os.path.join(folder_a_path, filename_a)
    img1 = load_image_cv(img1_path)

    img2_path = os.path.join(folder_b_path, filename_b)
    img2 = load_image_cv(img2_path)

    gt_image_path = os.path.join(label_path, filename_label)
    gt_image = io.imread(gt_image_path)

    img1_gray = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    img2_gray = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

    keypoints1, descriptors1 = brisk.detectAndCompute(img1_gray, None)
    keypoints2, descriptors2 = brisk.detectAndCompute(img2_gray, None)

    descriptors1_float = descriptors1.astype(np.float32)
    descriptors2_float = descriptors2.astype(np.float32)

    matches = flann.knnMatch(descriptors1_float, descriptors2_float, k=2)

    good_matches = []
    for m, n in matches:
        if m.distance < 0.85 * n.distance:
            good_matches.append(m)

    if len(good_matches) > 10:
        points1 = np.zeros((len(good_matches), 2), dtype=np.float32)
        points2 = np.zeros_like(points1)

        for i, match in enumerate(good_matches):
            points1[i, :] = keypoints1[match.queryIdx].pt
            points2[i, :] = keypoints2[match.trainIdx].pt

        M, mask = cv2.estimateAffinePartial2D(points1, points2, method=cv2.RANSAC)

        registered_img = cv2.warpAffine(img2, M, (img1.shape[1], img1.shape[0]))

        registered_img_gray = cv2.cvtColor(registered_img, cv2.COLOR_BGR2GRAY)

        edges = cv2.Canny(registered_img_gray, 200, 585)
        edges_binary = (edges > np.percentile(edges, 90)).astype(np.uint8)

        kernel = np.ones((5, 5), np.uint8)
        dilated_edges = cv2.dilate(edges, kernel, iterations=1)

        binary_change_map = dilated_edges > 0

        ssim_index = ssim(registered_img_gray, gt_image)
        mse_value = mse(registered_img_gray, gt_image)

        return ssim_index, mse_value

    else:
        return None, None

base_dir = '/content/drive/MyDrive/data/train'
folder_a_path = os.path.join(base_dir, 'A')
folder_b_path = os.path.join(base_dir, 'B')
label_path = os.path.join(base_dir, 'label')

folder_a_images = os.listdir(folder_a_path)
folder_b_images = os.listdir(folder_b_path)
label_images = os.listdir(label_path)

brisk = cv2.BRISK_create()

FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=3)
search_params = dict(checks=70)
flann = cv2.FlannBasedMatcher(index_params, search_params)

ssim_scores = []
mse_scores = []

# Create pool for multiprocessing
pool = Pool()

# Iterate through each image pair
for ssim_score, mse_score in pool.imap_unordered(process_image_pair, zip(folder_a_images, folder_b_images, label_images)):
    if ssim_score is not None:
        ssim_scores.append(ssim_score)
        mse_scores.append(mse_score)
    else:
        print("Not enough matches for an image pair.")

pool.close()
pool.join()

mean_ssim = np.mean(ssim_scores)
mean_mse = np.mean(mse_scores)

print(f"Mean SSIM score: {mean_ssim}")
print(f"Mean MSE score: {mean_mse}")


Mean SSIM score: 0.561227233190889
Mean MSE score: 5732.9314264511795


BRISK - SOBEL

In [ ]:
import os
import cv2
import numpy as np
from skimage import io
from skimage.metrics import structural_similarity as ssim, mean_squared_error as mse
import matplotlib.pyplot as plt
from multiprocessing import Pool

# Load image with OpenCV
def load_image_cv(path):
    return cv2.imread(path, cv2.IMREAD_COLOR)

def process_image_pair(args):
    filename_a, filename_b, filename_label = args
    img1_path = os.path.join(folder_a_path, filename_a)
    img1 = load_image_cv(img1_path)

    img2_path = os.path.join(folder_b_path, filename_b)
    img2 = load_image_cv(img2_path)

    gt_image_path = os.path.join(label_path, filename_label)
    gt_image = io.imread(gt_image_path)

    img1_gray = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    img2_gray = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

    keypoints1, descriptors1 = brisk.detectAndCompute(img1_gray, None)
    keypoints2, descriptors2 = brisk.detectAndCompute(img2_gray, None)

    descriptors1_float = descriptors1.astype(np.float32)
    descriptors2_float = descriptors2.astype(np.float32)

    matches = flann.knnMatch(descriptors1_float, descriptors2_float, k=2)

    good_matches = []
    for m, n in matches:
        if m.distance < 0.85 * n.distance:
            good_matches.append(m)

    if len(good_matches) > 10:
        points1 = np.zeros((len(good_matches), 2), dtype=np.float32)
        points2 = np.zeros_like(points1)

        for i, match in enumerate(good_matches):
            points1[i, :] = keypoints1[match.queryIdx].pt
            points2[i, :] = keypoints2[match.trainIdx].pt

        M, mask = cv2.estimateAffinePartial2D(points1, points2, method=cv2.RANSAC)

        registered_img = cv2.warpAffine(img2, M, (img1.shape[1], img1.shape[0]))

        registered_img_gray = cv2.cvtColor(registered_img, cv2.COLOR_BGR2GRAY)

        # Use Sobel operator for edge detection
        sobel_x = cv2.Sobel(registered_img_gray, cv2.CV_64F, 1, 0, ksize=3)
        sobel_y = cv2.Sobel(registered_img_gray, cv2.CV_64F, 0, 1, ksize=3)
        edges = np.sqrt(sobel_x**2 + sobel_y**2)
        edges_binary = (edges > np.percentile(edges, 90)).astype(np.uint8)

        kernel = np.ones((5, 5), np.uint8)
        dilated_edges = cv2.dilate(edges_binary, kernel, iterations=1)

        binary_change_map = dilated_edges > 0

        ssim_index = ssim(registered_img_gray, gt_image)
        mse_value = mse(registered_img_gray, gt_image)

        return ssim_index, mse_value

    else:
        return None, None

base_dir = '/content/drive/MyDrive/data/train'
folder_a_path = os.path.join(base_dir, 'A')
folder_b_path = os.path.join(base_dir, 'B')
label_path = os.path.join(base_dir, 'label')

folder_a_images = os.listdir(folder_a_path)
folder_b_images = os.listdir(folder_b_path)
label_images = os.listdir(label_path)

brisk = cv2.BRISK_create()

FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=3)
search_params = dict(checks=70)
flann = cv2.FlannBasedMatcher(index_params, search_params)

ssim_scores = []
mse_scores = []

# Create pool for multiprocessing
pool = Pool()

# Iterate through each image pair
for ssim_score, mse_score in pool.imap_unordered(process_image_pair, zip(folder_a_images, folder_b_images, label_images)):
    if ssim_score is not None:
        ssim_scores.append(ssim_score)
        mse_scores.append(mse_score)
    else:
        print("Not enough matches for an image pair.")

pool.close()
pool.join()

mean_ssim = np.mean(ssim_scores)
mean_mse = np.mean(mse_scores)

print(f"Mean SSIM score: {mean_ssim}")
print(f"Mean MSE score: {mean_mse}")


Mean SSIM score: 0.5895145157485331
Mean MSE score: 5374.1904741458675


BRISK - PREWITT

In [ ]:
import os
import cv2
import numpy as np
from skimage import io
from skimage.metrics import structural_similarity as ssim, mean_squared_error as mse
import matplotlib.pyplot as plt
from multiprocessing import Pool

# Load image with OpenCV
def load_image_cv(path):
    return cv2.imread(path, cv2.IMREAD_COLOR)

def process_image_pair(args):
    filename_a, filename_b, filename_label = args
    img1_path = os.path.join(folder_a_path, filename_a)
    img1 = load_image_cv(img1_path)

    img2_path = os.path.join(folder_b_path, filename_b)
    img2 = load_image_cv(img2_path)

    gt_image_path = os.path.join(label_path, filename_label)
    gt_image = io.imread(gt_image_path)

    img1_gray = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    img2_gray = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

    keypoints1, descriptors1 = brisk.detectAndCompute(img1_gray, None)
    keypoints2, descriptors2 = brisk.detectAndCompute(img2_gray, None)

    descriptors1_float = descriptors1.astype(np.float32)
    descriptors2_float = descriptors2.astype(np.float32)

    matches = flann.knnMatch(descriptors1_float, descriptors2_float, k=2)

    good_matches = []
    for m, n in matches:
        if m.distance < 0.85 * n.distance:
            good_matches.append(m)

    if len(good_matches) > 10:
        points1 = np.zeros((len(good_matches), 2), dtype=np.float32)
        points2 = np.zeros_like(points1)

        for i, match in enumerate(good_matches):
            points1[i, :] = keypoints1[match.queryIdx].pt
            points2[i, :] = keypoints2[match.trainIdx].pt

        M, mask = cv2.estimateAffinePartial2D(points1, points2, method=cv2.RANSAC)

        registered_img = cv2.warpAffine(img2, M, (img1.shape[1], img1.shape[0]))

        registered_img_gray = cv2.cvtColor(registered_img, cv2.COLOR_BGR2GRAY)

        # Use Prewitt operator for edge detection
        prewitt_x = cv2.Sobel(registered_img_gray, cv2.CV_64F, 1, 0, ksize=3)
        prewitt_y = cv2.Sobel(registered_img_gray, cv2.CV_64F, 0, 1, ksize=3)
        edges = np.sqrt(prewitt_x**2 + prewitt_y**2)
        edges_binary = (edges > np.percentile(edges, 90)).astype(np.uint8)

        kernel = np.ones((5, 5), np.uint8)
        dilated_edges = cv2.dilate(edges_binary, kernel, iterations=1)

        binary_change_map = dilated_edges > 0

        ssim_index = ssim(registered_img_gray, gt_image)
        mse_value = mse(registered_img_gray, gt_image)

        return ssim_index, mse_value

    else:
        return None, None

base_dir = '/content/drive/MyDrive/data/train'
folder_a_path = os.path.join(base_dir, 'A')
folder_b_path = os.path.join(base_dir, 'B')
label_path = os.path.join(base_dir, 'label')

folder_a_images = os.listdir(folder_a_path)
folder_b_images = os.listdir(folder_b_path)
label_images = os.listdir(label_path)

brisk = cv2.BRISK_create()

FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=3)
search_params = dict(checks=70)
flann = cv2.FlannBasedMatcher(index_params, search_params)

ssim_scores = []
mse_scores = []

# Create pool for multiprocessing
pool = Pool()

# Iterate through each image pair
for ssim_score, mse_score in pool.imap_unordered(process_image_pair, zip(folder_a_images, folder_b_images, label_images)):
    if ssim_score is not None:
        ssim_scores.append(ssim_score)
        mse_scores.append(mse_score)
    else:
        print("Not enough matches for an image pair.")

pool.close()
pool.join()

mean_ssim = np.mean(ssim_scores)
mean_mse = np.mean(mse_scores)

print(f"Mean SSIM score: {mean_ssim}")
print(f"Mean MSE score: {mean_mse}")


Mean SSIM score: 0.5761191611681542
Mean MSE score: 5623.25329404038


BRISK - LOG

In [ ]:
import os
import cv2
import numpy as np
from skimage import io
from skimage.metrics import structural_similarity as ssim, mean_squared_error as mse
import matplotlib.pyplot as plt
from multiprocessing import Pool

# Load image with OpenCV
def load_image_cv(path):
    return cv2.imread(path, cv2.IMREAD_COLOR)

def process_image_pair(args):
    filename_a, filename_b, filename_label = args
    img1_path = os.path.join(folder_a_path, filename_a)
    img1 = load_image_cv(img1_path)

    img2_path = os.path.join(folder_b_path, filename_b)
    img2 = load_image_cv(img2_path)

    gt_image_path = os.path.join(label_path, filename_label)
    gt_image = io.imread(gt_image_path)

    img1_gray = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    img2_gray = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

    keypoints1, descriptors1 = brisk.detectAndCompute(img1_gray, None)
    keypoints2, descriptors2 = brisk.detectAndCompute(img2_gray, None)

    descriptors1_float = descriptors1.astype(np.float32)
    descriptors2_float = descriptors2.astype(np.float32)

    matches = flann.knnMatch(descriptors1_float, descriptors2_float, k=2)

    good_matches = []
    for m, n in matches:
        if m.distance < 0.85 * n.distance:
            good_matches.append(m)

    if len(good_matches) > 10:
        points1 = np.zeros((len(good_matches), 2), dtype=np.float32)
        points2 = np.zeros_like(points1)

        for i, match in enumerate(good_matches):
            points1[i, :] = keypoints1[match.queryIdx].pt
            points2[i, :] = keypoints2[match.trainIdx].pt

        M, mask = cv2.estimateAffinePartial2D(points1, points2, method=cv2.RANSAC)

        registered_img = cv2.warpAffine(img2, M, (img1.shape[1], img1.shape[0]))

        registered_img_gray = cv2.cvtColor(registered_img, cv2.COLOR_BGR2GRAY)

        # Use Laplacian of Gaussian (LOG) operator for edge detection
        edges = cv2.Laplacian(registered_img_gray, cv2.CV_64F)
        edges_binary = (edges > np.percentile(edges, 90)).astype(np.uint8)

        kernel = np.ones((5, 5), np.uint8)
        dilated_edges = cv2.dilate(edges_binary, kernel, iterations=1)

        binary_change_map = dilated_edges > 0

        ssim_index = ssim(registered_img_gray, gt_image)
        mse_value = mse(registered_img_gray, gt_image)

        return ssim_index, mse_value

    else:
        return None, None

base_dir = '/content/drive/MyDrive/data/train'
folder_a_path = os.path.join(base_dir, 'A')
folder_b_path = os.path.join(base_dir, 'B')
label_path = os.path.join(base_dir, 'label')

folder_a_images = os.listdir(folder_a_path)
folder_b_images = os.listdir(folder_b_path)
label_images = os.listdir(label_path)

brisk = cv2.BRISK_create()

FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=3)
search_params = dict(checks=70)
flann = cv2.FlannBasedMatcher(index_params, search_params)

ssim_scores = []
mse_scores = []

# Create pool for multiprocessing
pool = Pool()

# Iterate through each image pair
for ssim_score, mse_score in pool.imap_unordered(process_image_pair, zip(folder_a_images, folder_b_images, label_images)):
    if ssim_score is not None:
        ssim_scores.append(ssim_score)
        mse_scores.append(mse_score)
    else:
        print("Not enough matches for an image pair.")

pool.close()
pool.join()

mean_ssim = np.mean(ssim_scores)
mean_mse = np.mean(mse_scores)

print(f"Mean SSIM score: {mean_ssim}")
print(f"Mean MSE score: {mean_mse}")


Mean SSIM score: 0.5829774264858655
Mean MSE score: 5493.61226380809


BRISK - SCHAAR

In [ ]:
import os
import cv2
import numpy as np
from skimage import io
from skimage.metrics import structural_similarity as ssim, mean_squared_error as mse
import matplotlib.pyplot as plt
from multiprocessing import Pool

# Load image with OpenCV
def load_image_cv(path):
    return cv2.imread(path, cv2.IMREAD_COLOR)

def process_image_pair(args):
    filename_a, filename_b, filename_label = args
    img1_path = os.path.join(folder_a_path, filename_a)
    img1 = load_image_cv(img1_path)

    img2_path = os.path.join(folder_b_path, filename_b)
    img2 = load_image_cv(img2_path)

    gt_image_path = os.path.join(label_path, filename_label)
    gt_image = io.imread(gt_image_path)

    img1_gray = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    img2_gray = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

    keypoints1, descriptors1 = brisk.detectAndCompute(img1_gray, None)
    keypoints2, descriptors2 = brisk.detectAndCompute(img2_gray, None)

    descriptors1_float = descriptors1.astype(np.float32)
    descriptors2_float = descriptors2.astype(np.float32)

    matches = flann.knnMatch(descriptors1_float, descriptors2_float, k=2)

    good_matches = []
    for m, n in matches:
        if m.distance < 0.85 * n.distance:
            good_matches.append(m)

    if len(good_matches) > 10:
        points1 = np.zeros((len(good_matches), 2), dtype=np.float32)
        points2 = np.zeros_like(points1)

        for i, match in enumerate(good_matches):
            points1[i, :] = keypoints1[match.queryIdx].pt
            points2[i, :] = keypoints2[match.trainIdx].pt

        M, mask = cv2.estimateAffinePartial2D(points1, points2, method=cv2.RANSAC)

        registered_img = cv2.warpAffine(img2, M, (img1.shape[1], img1.shape[0]))

        registered_img_gray = cv2.cvtColor(registered_img, cv2.COLOR_BGR2GRAY)

        # Use Scharr operator for edge detection
        scharr_x = cv2.Scharr(registered_img_gray, cv2.CV_64F, 1, 0)
        scharr_y = cv2.Scharr(registered_img_gray, cv2.CV_64F, 0, 1)
        edges = np.sqrt(scharr_x**2 + scharr_y**2)
        edges_binary = (edges > np.percentile(edges, 90)).astype(np.uint8)

        kernel = np.ones((5, 5), np.uint8)
        dilated_edges = cv2.dilate(edges_binary, kernel, iterations=1)

        binary_change_map = dilated_edges > 0

        ssim_index = ssim(registered_img_gray, gt_image)
        mse_value = mse(registered_img_gray, gt_image)

        return ssim_index, mse_value

    else:
        return None, None

base_dir = '/content/drive/MyDrive/data/train'
folder_a_path = os.path.join(base_dir, 'A')
folder_b_path = os.path.join(base_dir, 'B')
label_path = os.path.join(base_dir, 'label')

folder_a_images = os.listdir(folder_a_path)
folder_b_images = os.listdir(folder_b_path)
label_images = os.listdir(label_path)

brisk = cv2.BRISK_create()

FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=3)
search_params = dict(checks=70)
flann = cv2.FlannBasedMatcher(index_params, search_params)

ssim_scores = []
mse_scores = []

# Create pool for multiprocessing
pool = Pool()

# Iterate through each image pair
for ssim_score, mse_score in pool.imap_unordered(process_image_pair, zip(folder_a_images, folder_b_images, label_images)):
    if ssim_score is not None:
        ssim_scores.append(ssim_score)
        mse_scores.append(mse_score)
    else:
        print("Not enough matches for an image pair.")

pool.close()
pool.join()

mean_ssim = np.mean(ssim_scores)
mean_mse = np.mean(mse_scores)

print(f"Mean SSIM score: {mean_ssim}")
print(f"Mean MSE score: {mean_mse}")


Mean SSIM score: 0.5364655800114982
Mean MSE score: 5737.0052035149565


ORB - CANNY

In [ ]:
import os
import cv2
import numpy as np
from skimage import io
from skimage.metrics import structural_similarity as ssim, mean_squared_error as mse
import matplotlib.pyplot as plt
from multiprocessing import Pool

# Load image with OpenCV
def load_image_cv(path):
    return cv2.imread(path, cv2.IMREAD_COLOR)

def process_image_pair(args):
    filename_a, filename_b, filename_label = args
    img1_path = os.path.join(folder_a_path, filename_a)
    img1 = load_image_cv(img1_path)

    img2_path = os.path.join(folder_b_path, filename_b)
    img2 = load_image_cv(img2_path)

    gt_image_path = os.path.join(label_path, filename_label)
    gt_image = io.imread(gt_image_path)

    img1_gray = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    img2_gray = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

    orb = cv2.ORB_create()

    keypoints1, descriptors1 = orb.detectAndCompute(img1_gray, None)
    keypoints2, descriptors2 = orb.detectAndCompute(img2_gray, None)

    descriptors1_float = descriptors1.astype(np.float32)
    descriptors2_float = descriptors2.astype(np.float32)

    matches = flann.knnMatch(descriptors1_float, descriptors2_float, k=2)

    good_matches = []
    for m, n in matches:
        if m.distance < 0.85 * n.distance:
            good_matches.append(m)

    if len(good_matches) > 10:
        points1 = np.zeros((len(good_matches), 2), dtype=np.float32)
        points2 = np.zeros_like(points1)

        for i, match in enumerate(good_matches):
            points1[i, :] = keypoints1[match.queryIdx].pt
            points2[i, :] = keypoints2[match.trainIdx].pt

        M, mask = cv2.estimateAffinePartial2D(points1, points2, method=cv2.RANSAC)

        registered_img = cv2.warpAffine(img2, M, (img1.shape[1], img1.shape[0]))

        registered_img_gray = cv2.cvtColor(registered_img, cv2.COLOR_BGR2GRAY)

        edges = cv2.Canny(registered_img_gray, 200, 585)
        edges_binary = (edges > np.percentile(edges, 90)).astype(np.uint8)

        kernel = np.ones((5, 5), np.uint8)
        dilated_edges = cv2.dilate(edges, kernel, iterations=1)

        binary_change_map = dilated_edges > 0

        ssim_index = ssim(registered_img_gray, gt_image)
        mse_value = mse(registered_img_gray, gt_image)

        return ssim_index, mse_value

    else:
        return None, None

base_dir = '/content/drive/MyDrive/data/train'
folder_a_path = os.path.join(base_dir, 'A')
folder_b_path = os.path.join(base_dir, 'B')
label_path = os.path.join(base_dir, 'label')

folder_a_images = os.listdir(folder_a_path)
folder_b_images = os.listdir(folder_b_path)
label_images = os.listdir(label_path)

FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=3)
search_params = dict(checks=70)
flann = cv2.FlannBasedMatcher(index_params, search_params)

ssim_scores = []
mse_scores = []

# Create pool for multiprocessing
pool = Pool()

# Iterate through each image pair
for ssim_score, mse_score in pool.imap_unordered(process_image_pair, zip(folder_a_images, folder_b_images, label_images)):
    if ssim_score is not None:
        ssim_scores.append(ssim_score)
        mse_scores.append(mse_score)
    else:
        print("Not enough matches for an image pair.")

pool.close()
pool.join()

mean_ssim = np.mean(ssim_scores)
mean_mse = np.mean(mse_scores)

print(f"Mean SSIM score: {mean_ssim}")
print(f"Mean MSE score: {mean_mse}")


Mean SSIM score: 0.6638272652354902
Mean MSE score: 4790.527142421851


ORB - SOBEL

In [ ]:
import os
import cv2
import numpy as np
from skimage import io
from skimage.metrics import structural_similarity as ssim, mean_squared_error as mse
import matplotlib.pyplot as plt
from multiprocessing import Pool

# Load image with OpenCV
def load_image_cv(path):
    return cv2.imread(path, cv2.IMREAD_COLOR)

def process_image_pair(args):
    filename_a, filename_b, filename_label = args
    img1_path = os.path.join(folder_a_path, filename_a)
    img1 = load_image_cv(img1_path)

    img2_path = os.path.join(folder_b_path, filename_b)
    img2 = load_image_cv(img2_path)

    gt_image_path = os.path.join(label_path, filename_label)
    gt_image = io.imread(gt_image_path)

    img1_gray = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    img2_gray = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

    orb = cv2.ORB_create()

    keypoints1, descriptors1 = orb.detectAndCompute(img1_gray, None)
    keypoints2, descriptors2 = orb.detectAndCompute(img2_gray, None)

    descriptors1_float = descriptors1.astype(np.float32)
    descriptors2_float = descriptors2.astype(np.float32)

    matches = flann.knnMatch(descriptors1_float, descriptors2_float, k=2)

    good_matches = []
    for m, n in matches:
        if m.distance < 0.85 * n.distance:
            good_matches.append(m)

    if len(good_matches) > 10:
        points1 = np.zeros((len(good_matches), 2), dtype=np.float32)
        points2 = np.zeros_like(points1)

        for i, match in enumerate(good_matches):
            points1[i, :] = keypoints1[match.queryIdx].pt
            points2[i, :] = keypoints2[match.trainIdx].pt

        M, mask = cv2.estimateAffinePartial2D(points1, points2, method=cv2.RANSAC)

        registered_img = cv2.warpAffine(img2, M, (img1.shape[1], img1.shape[0]))

        registered_img_gray = cv2.cvtColor(registered_img, cv2.COLOR_BGR2GRAY)

        # Sobel edge detection
        sobel_x = cv2.Sobel(registered_img_gray, cv2.CV_64F, 1, 0, ksize=5)
        sobel_y = cv2.Sobel(registered_img_gray, cv2.CV_64F, 0, 1, ksize=5)
        sobel_edges = np.sqrt(sobel_x**2 + sobel_y**2)

        sobel_binary_edges = (sobel_edges > np.percentile(sobel_edges, 90)).astype(np.uint8)

        kernel = np.ones((5, 5), np.uint8)
        dilated_edges = cv2.dilate(sobel_binary_edges, kernel, iterations=1)

        binary_change_map = dilated_edges > 0

        ssim_index = ssim(registered_img_gray, gt_image)
        mse_value = mse(registered_img_gray, gt_image)

        return ssim_index, mse_value

    else:
        return None, None

base_dir = '/content/drive/MyDrive/data/train'
folder_a_path = os.path.join(base_dir, 'A')
folder_b_path = os.path.join(base_dir, 'B')
label_path = os.path.join(base_dir, 'label')

folder_a_images = os.listdir(folder_a_path)
folder_b_images = os.listdir(folder_b_path)
label_images = os.listdir(label_path)

FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=3)
search_params = dict(checks=70)
flann = cv2.FlannBasedMatcher(index_params, search_params)

ssim_scores = []
mse_scores = []

# Create pool for multiprocessing
pool = Pool()

# Iterate through each image pair
for ssim_score, mse_score in pool.imap_unordered(process_image_pair, zip(folder_a_images, folder_b_images, label_images)):
    if ssim_score is not None:
        ssim_scores.append(ssim_score)
        mse_scores.append(mse_score)
    else:
        print("Not enough matches for an image pair.")

pool.close()
pool.join()

mean_ssim = np.mean(ssim_scores)
mean_mse = np.mean(mse_scores)

print(f"Mean SSIM score: {mean_ssim}")
print(f"Mean MSE score: {mean_mse}")


Mean SSIM score: 0.6361657128316988
Mean MSE score: 5209.105129972737


ORB - PREWITT

In [ ]:
import os
import cv2
import numpy as np
from skimage import io
from skimage.metrics import structural_similarity as ssim, mean_squared_error as mse
import matplotlib.pyplot as plt
from multiprocessing import Pool

# Load image with OpenCV
def load_image_cv(path):
    return cv2.imread(path, cv2.IMREAD_COLOR)

def prewitt_operator(img):
    kernel_x = np.array([[1, 0, -1],
                         [1, 0, -1],
                         [1, 0, -1]])

    kernel_y = np.array([[1, 1, 1],
                         [0, 0, 0],
                         [-1, -1, -1]])

    prewitt_x = cv2.filter2D(img, -1, kernel_x)
    prewitt_y = cv2.filter2D(img, -1, kernel_y)

    return np.sqrt(prewitt_x**2 + prewitt_y**2)

def process_image_pair(args):
    filename_a, filename_b, filename_label = args
    img1_path = os.path.join(folder_a_path, filename_a)
    img1 = load_image_cv(img1_path)

    img2_path = os.path.join(folder_b_path, filename_b)
    img2 = load_image_cv(img2_path)

    gt_image_path = os.path.join(label_path, filename_label)
    gt_image = io.imread(gt_image_path)

    img1_gray = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    img2_gray = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

    orb = cv2.ORB_create()

    keypoints1, descriptors1 = orb.detectAndCompute(img1_gray, None)
    keypoints2, descriptors2 = orb.detectAndCompute(img2_gray, None)

    descriptors1_float = descriptors1.astype(np.float32)
    descriptors2_float = descriptors2.astype(np.float32)

    matches = flann.knnMatch(descriptors1_float, descriptors2_float, k=2)

    good_matches = []
    for m, n in matches:
        if m.distance < 0.85 * n.distance:
            good_matches.append(m)

    if len(good_matches) > 10:
        points1 = np.zeros((len(good_matches), 2), dtype=np.float32)
        points2 = np.zeros_like(points1)

        for i, match in enumerate(good_matches):
            points1[i, :] = keypoints1[match.queryIdx].pt
            points2[i, :] = keypoints2[match.trainIdx].pt

        M, mask = cv2.estimateAffinePartial2D(points1, points2, method=cv2.RANSAC)

        registered_img = cv2.warpAffine(img2, M, (img1.shape[1], img1.shape[0]))

        registered_img_gray = cv2.cvtColor(registered_img, cv2.COLOR_BGR2GRAY)

        # Prewitt edge detection
        prewitt_edges = prewitt_operator(registered_img_gray)
        prewitt_binary_edges = (prewitt_edges > np.percentile(prewitt_edges, 90)).astype(np.uint8)

        kernel = np.ones((5, 5), np.uint8)
        dilated_edges = cv2.dilate(prewitt_binary_edges, kernel, iterations=1)

        binary_change_map = dilated_edges > 0

        ssim_index = ssim(registered_img_gray, gt_image)
        mse_value = mse(registered_img_gray, gt_image)

        return ssim_index, mse_value

    else:
        return None, None

base_dir = '/content/drive/MyDrive/data/train'
folder_a_path = os.path.join(base_dir, 'A')
folder_b_path = os.path.join(base_dir, 'B')
label_path = os.path.join(base_dir, 'label')

folder_a_images = os.listdir(folder_a_path)
folder_b_images = os.listdir(folder_b_path)
label_images = os.listdir(label_path)

FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=3)
search_params = dict(checks=70)
flann = cv2.FlannBasedMatcher(index_params, search_params)

ssim_scores = []
mse_scores = []

# Create pool for multiprocessing
pool = Pool()

# Iterate through each image pair
for ssim_score, mse_score in pool.imap_unordered(process_image_pair, zip(folder_a_images, folder_b_images, label_images)):
    if ssim_score is not None:
        ssim_scores.append(ssim_score)
        mse_scores.append(mse_score)
    else:
        print("Not enough matches for an image pair.")

pool.close()
pool.join()

mean_ssim = np.mean(ssim_scores)
mean_mse = np.mean(mse_scores)

print(f"Mean SSIM score: {mean_ssim}")
print(f"Mean MSE score: {mean_mse}")


Mean SSIM score: 0.6612250293095355
Mean MSE score: 4853.589983243621


ORB - LOG

In [ ]:
import os
import cv2
import numpy as np
from skimage import io
from skimage.metrics import structural_similarity as ssim, mean_squared_error as mse
import matplotlib.pyplot as plt
from multiprocessing import Pool

# Load image with OpenCV
def load_image_cv(path):
    return cv2.imread(path, cv2.IMREAD_COLOR)

def process_image_pair(args):
    filename_a, filename_b, filename_label = args
    img1_path = os.path.join(folder_a_path, filename_a)
    img1 = load_image_cv(img1_path)

    img2_path = os.path.join(folder_b_path, filename_b)
    img2 = load_image_cv(img2_path)

    gt_image_path = os.path.join(label_path, filename_label)
    gt_image = io.imread(gt_image_path)

    img1_gray = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    img2_gray = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

    orb = cv2.ORB_create()

    keypoints1, descriptors1 = orb.detectAndCompute(img1_gray, None)
    keypoints2, descriptors2 = orb.detectAndCompute(img2_gray, None)

    descriptors1_float = descriptors1.astype(np.float32)
    descriptors2_float = descriptors2.astype(np.float32)

    matches = flann.knnMatch(descriptors1_float, descriptors2_float, k=2)

    good_matches = []
    for m, n in matches:
        if m.distance < 0.85 * n.distance:
            good_matches.append(m)

    if len(good_matches) > 10:
        points1 = np.zeros((len(good_matches), 2), dtype=np.float32)
        points2 = np.zeros_like(points1)

        for i, match in enumerate(good_matches):
            points1[i, :] = keypoints1[match.queryIdx].pt
            points2[i, :] = keypoints2[match.trainIdx].pt

        M, mask = cv2.estimateAffinePartial2D(points1, points2, method=cv2.RANSAC)

        registered_img = cv2.warpAffine(img2, M, (img1.shape[1], img1.shape[0]))

        registered_img_gray = cv2.cvtColor(registered_img, cv2.COLOR_BGR2GRAY)

        # Apply Gaussian blur
        blurred_img = cv2.GaussianBlur(registered_img_gray, (5, 5), 0)

        # Apply Laplacian of Gaussian (LoG) operator
        log_edges = cv2.Laplacian(blurred_img, cv2.CV_64F)

        log_binary_edges = (log_edges > np.percentile(log_edges, 90)).astype(np.uint8)

        kernel = np.ones((5, 5), np.uint8)
        dilated_edges = cv2.dilate(log_binary_edges, kernel, iterations=1)

        binary_change_map = dilated_edges > 0

        ssim_index = ssim(registered_img_gray, gt_image)
        mse_value = mse(registered_img_gray, gt_image)

        return ssim_index, mse_value

    else:
        return None, None

base_dir = '/content/drive/MyDrive/data/train'
folder_a_path = os.path.join(base_dir, 'A')
folder_b_path = os.path.join(base_dir, 'B')
label_path = os.path.join(base_dir, 'label')

folder_a_images = os.listdir(folder_a_path)
folder_b_images = os.listdir(folder_b_path)
label_images = os.listdir(label_path)

FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=3)
search_params = dict(checks=70)
flann = cv2.FlannBasedMatcher(index_params, search_params)

ssim_scores = []
mse_scores = []

# Create pool for multiprocessing
pool = Pool()

# Iterate through each image pair
for ssim_score, mse_score in pool.imap_unordered(process_image_pair, zip(folder_a_images, folder_b_images, label_images)):
    if ssim_score is not None:
        ssim_scores.append(ssim_score)
        mse_scores.append(mse_score)
    else:
        print("Not enough matches for an image pair.")

pool.close()
pool.join()

mean_ssim = np.mean(ssim_scores)
mean_mse = np.mean(mse_scores)

print(f"Mean SSIM score: {mean_ssim}")
print(f"Mean MSE score: {mean_mse}")


Mean SSIM score: 0.6424586983995815
Mean MSE score: 5087.835956637779


ORB - SCHAAR

In [ ]:
import os
import cv2
import numpy as np
from skimage import io
from skimage.metrics import structural_similarity as ssim, mean_squared_error as mse
import matplotlib.pyplot as plt
from multiprocessing import Pool

# Load image with OpenCV
def load_image_cv(path):
    return cv2.imread(path, cv2.IMREAD_COLOR)

def process_image_pair(args):
    filename_a, filename_b, filename_label = args
    img1_path = os.path.join(folder_a_path, filename_a)
    img1 = load_image_cv(img1_path)

    img2_path = os.path.join(folder_b_path, filename_b)
    img2 = load_image_cv(img2_path)

    gt_image_path = os.path.join(label_path, filename_label)
    gt_image = io.imread(gt_image_path)

    img1_gray = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    img2_gray = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

    orb = cv2.ORB_create()

    keypoints1, descriptors1 = orb.detectAndCompute(img1_gray, None)
    keypoints2, descriptors2 = orb.detectAndCompute(img2_gray, None)

    descriptors1_float = descriptors1.astype(np.float32)
    descriptors2_float = descriptors2.astype(np.float32)

    matches = flann.knnMatch(descriptors1_float, descriptors2_float, k=2)

    good_matches = []
    for m, n in matches:
        if m.distance < 0.85 * n.distance:
            good_matches.append(m)

    if len(good_matches) > 10:
        points1 = np.zeros((len(good_matches), 2), dtype=np.float32)
        points2 = np.zeros_like(points1)

        for i, match in enumerate(good_matches):
            points1[i, :] = keypoints1[match.queryIdx].pt
            points2[i, :] = keypoints2[match.trainIdx].pt

        M, mask = cv2.estimateAffinePartial2D(points1, points2, method=cv2.RANSAC)

        registered_img = cv2.warpAffine(img2, M, (img1.shape[1], img1.shape[0]))

        registered_img_gray = cv2.cvtColor(registered_img, cv2.COLOR_BGR2GRAY)

        # Apply Scharr operator for edge detection
        scharr_x = cv2.Scharr(registered_img_gray, cv2.CV_64F, 1, 0)
        scharr_y = cv2.Scharr(registered_img_gray, cv2.CV_64F, 0, 1)
        scharr_edges = np.sqrt(scharr_x**2 + scharr_y**2)

        scharr_binary_edges = (scharr_edges > np.percentile(scharr_edges, 90)).astype(np.uint8)

        kernel = np.ones((5, 5), np.uint8)
        dilated_edges = cv2.dilate(scharr_binary_edges, kernel, iterations=1)

        binary_change_map = dilated_edges > 0

        ssim_index = ssim(registered_img_gray, gt_image)
        mse_value = mse(registered_img_gray, gt_image)

        return ssim_index, mse_value

    else:
        return None, None

base_dir = '/content/drive/MyDrive/data/train'
folder_a_path = os.path.join(base_dir, 'A')
folder_b_path = os.path.join(base_dir, 'B')
label_path = os.path.join(base_dir, 'label')

folder_a_images = os.listdir(folder_a_path)
folder_b_images = os.listdir(folder_b_path)
label_images = os.listdir(label_path)

FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=3)
search_params = dict(checks=70)
flann = cv2.FlannBasedMatcher(index_params, search_params)

ssim_scores = []
mse_scores = []

# Create pool for multiprocessing
pool = Pool()

# Iterate through each image pair
for ssim_score, mse_score in pool.imap_unordered(process_image_pair, zip(folder_a_images, folder_b_images, label_images)):
    if ssim_score is not None:
        ssim_scores.append(ssim_score)
        mse_scores.append(mse_score)
    else:
        print("Not enough matches for an image pair.")

pool.close()
pool.join()

mean_ssim = np.mean(ssim_scores)
mean_mse = np.mean(mse_scores)

print(f"Mean SSIM score: {mean_ssim}")
print(f"Mean MSE score: {mean_mse}")


Mean SSIM score: 0.6473816073386206
Mean MSE score: 5022.703651093901


In [ ]:
import os
import cv2
import numpy as np
from skimage import io
from skimage.metrics import structural_similarity as ssim, mean_squared_error as mse

# Load image with OpenCV
def load_image_cv(path):
    return cv2.imread(path, cv2.IMREAD_COLOR)

def process_image_pair(filename_a, filename_b, filename_label):
    img1_path = os.path.join(folder_a_path, filename_a)
    img1 = load_image_cv(img1_path)

    img2_path = os.path.join(folder_b_path, filename_b)
    img2 = load_image_cv(img2_path)

    gt_image_path = os.path.join(label_path, filename_label)
    gt_image = io.imread(gt_image_path)

    img1_gray = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    img2_gray = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

    # Create SIFT detector
    sift = cv2.SIFT_create()

    # Detect keypoints and compute descriptors
    keypoints1, descriptors1 = sift.detectAndCompute(img1_gray, None)
    keypoints2, descriptors2 = sift.detectAndCompute(img2_gray, None)

    # FLANN parameters
    FLANN_INDEX_KDTREE = 1
    index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=3)
    search_params = dict(checks=70)
    flann = cv2.FlannBasedMatcher(index_params, search_params)

    # Match descriptors using FLANN
    matches = flann.knnMatch(descriptors1, descriptors2, k=2)

    # Ratio test to find good matches
    good_matches = []
    for m, n in matches:
        if m.distance < 0.75 * n.distance:
            good_matches.append(m)

    if len(good_matches) > 10:
        # Extract keypoints for good matches
        points1 = np.float32([keypoints1[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
        points2 = np.float32([keypoints2[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)

        # Find homography using RANSAC
        M, mask = cv2.findHomography(points2, points1, cv2.RANSAC, 5.0)

        # Warp image2 to image1 using homography
        registered_img = cv2.warpPerspective(img2, M, (img1.shape[1], img1.shape[0]))

        # Convert registered image to grayscale
        registered_img_gray = cv2.cvtColor(registered_img, cv2.COLOR_BGR2GRAY)

        # Apply edge detection using Canny
        edges_registered = cv2.Canny(registered_img_gray, 200, 585)

        # Compute SSIM and MSE
        ssim_index = ssim(edges_registered, gt_image)
        mse_value = mse(edges_registered, gt_image)

        return ssim_index, mse_value

    else:
        return None, None

base_dir = '/content/drive/MyDrive/data/train'
folder_a_path = os.path.join(base_dir, 'A')
folder_b_path = os.path.join(base_dir, 'B')
label_path = os.path.join(base_dir, 'label')

folder_a_images = os.listdir(folder_a_path)
folder_b_images = os.listdir(folder_b_path)
label_images = os.listdir(label_path)

ssim_scores = []
mse_scores = []

# Iterate through each image pair
for filename_a, filename_b, filename_label in zip(folder_a_images, folder_b_images, label_images):
    ssim_score, mse_score = process_image_pair(filename_a, filename_b, filename_label)
    if ssim_score is not None:
        ssim_scores.append(ssim_score)
        mse_scores.append(mse_score)
    else:
        print("Not enough matches for an image pair.")

mean_ssim = np.mean(ssim_scores)
mean_mse = np.mean(mse_scores)

print(f"Mean SSIM score: {mean_ssim}")
print(f"Mean MSE score: {mean_mse}")


Not enough matches for an image pair.
Not enough matches for an image pair.
Not enough matches for an image pair.
Not enough matches for an image pair.
Not enough matches for an image pair.
Not enough matches for an image pair.
Not enough matches for an image pair.
Not enough matches for an image pair.
Not enough matches for an image pair.
Mean SSIM score: 0.8750486043485063
Mean MSE score: 3601.588294626376


In [ ]:
import os
import cv2
import numpy as np
from skimage import io
from skimage.metrics import structural_similarity as ssim, mean_squared_error as mse

# Load image with OpenCV
def load_image_cv(path):
    return cv2.imread(path, cv2.IMREAD_COLOR)

def process_image_pair(filename_a, filename_b, filename_label):
    img1_path = os.path.join(folder_a_path, filename_a)
    img1 = load_image_cv(img1_path)

    img2_path = os.path.join(folder_b_path, filename_b)
    img2 = load_image_cv(img2_path)

    gt_image_path = os.path.join(label_path, filename_label)
    gt_image = io.imread(gt_image_path)

    img1_gray = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    img2_gray = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

    # Create SIFT detector
    sift = cv2.SIFT_create()

    # Detect keypoints and compute descriptors
    keypoints1, descriptors1 = sift.detectAndCompute(img1_gray, None)
    keypoints2, descriptors2 = sift.detectAndCompute(img2_gray, None)

    # FLANN parameters
    FLANN_INDEX_KDTREE = 1
    index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=7)
    search_params = dict(checks=40)
    flann = cv2.FlannBasedMatcher(index_params, search_params)

    # Match descriptors using FLANN
    matches = flann.knnMatch(descriptors1, descriptors2, k=2)

    # Ratio test to find good matches
    good_matches = []
    for m, n in matches:
        if m.distance < 0.80 * n.distance:
            good_matches.append(m)

    if len(good_matches) > 10:
        # Extract keypoints for good matches
        points1 = np.float32([keypoints1[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
        points2 = np.float32([keypoints2[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)

        # Find homography using RANSAC
        M, mask = cv2.findHomography(points2, points1, cv2.RANSAC, 5.0)

        # Warp image2 to image1 using homography
        registered_img = cv2.warpPerspective(img2, M, (img1.shape[1], img1.shape[0]))

        # Convert registered image to grayscale
        registered_img_gray = cv2.cvtColor(registered_img, cv2.COLOR_BGR2GRAY)

        # Apply edge detection using Sobel
        sobel_x = cv2.Sobel(registered_img_gray, cv2.CV_64F, 1, 0, ksize=7)
        sobel_y = cv2.Sobel(registered_img_gray, cv2.CV_64F, 0, 1, ksize=7)
        edges_registered = np.sqrt(sobel_x**2 + sobel_y**2)

        # Convert edges_registered to the same data type as gt_image
        edges_registered = edges_registered.astype(gt_image.dtype)

        # Compute SSIM and MSE
        ssim_index = ssim(edges_registered, gt_image)
        mse_value = mse(edges_registered, gt_image)

        return ssim_index, mse_value

    else:
        return None, None

base_dir = '/content/drive/MyDrive/data/train'
folder_a_path = os.path.join(base_dir, 'A')
folder_b_path = os.path.join(base_dir, 'B')
label_path = os.path.join(base_dir, 'label')

folder_a_images = os.listdir(folder_a_path)
folder_b_images = os.listdir(folder_b_path)
label_images = os.listdir(label_path)

ssim_scores = []
mse_scores = []

# Iterate through each image pair
for filename_a, filename_b, filename_label in zip(folder_a_images, folder_b_images, label_images):
    ssim_score, mse_score = process_image_pair(filename_a, filename_b, filename_label)
    if ssim_score is not None:
        ssim_scores.append(ssim_score)
        mse_scores.append(mse_score)
    else:
        print("Not enough matches for an image pair.")

mean_ssim = np.mean(ssim_scores)
mean_mse = np.mean(mse_scores)

print(f"Mean SSIM score: {mean_ssim}")
print(f"Mean MSE score: {mean_mse}")


Mean SSIM score: 0.29712164975574495
Mean MSE score: 15220.920973070552


In [ ]:
import os
import cv2
import numpy as np
from skimage import io
from skimage.metrics import structural_similarity as ssim, mean_squared_error as mse

# Load image with OpenCV
def load_image_cv(path):
    return cv2.imread(path, cv2.IMREAD_COLOR)

def process_image_pair(filename_a, filename_b, filename_label):
    img1_path = os.path.join(folder_a_path, filename_a)
    img1 = load_image_cv(img1_path)

    img2_path = os.path.join(folder_b_path, filename_b)
    img2 = load_image_cv(img2_path)

    gt_image_path = os.path.join(label_path, filename_label)
    gt_image = io.imread(gt_image_path)

    img1_gray = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    img2_gray = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

    # Create SIFT detector
    sift = cv2.SIFT_create()

    # Detect keypoints and compute descriptors
    keypoints1, descriptors1 = sift.detectAndCompute(img1_gray, None)
    keypoints2, descriptors2 = sift.detectAndCompute(img2_gray, None)

    # FLANN parameters
    FLANN_INDEX_KDTREE = 1
    index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=3)
    search_params = dict(checks=70)
    flann = cv2.FlannBasedMatcher(index_params, search_params)

    # Match descriptors using FLANN
    matches = flann.knnMatch(descriptors1, descriptors2, k=2)

    # Ratio test to find good matches
    good_matches = []
    for m, n in matches:
        if m.distance < 0.75 * n.distance:
            good_matches.append(m)

    if len(good_matches) > 10:
        # Extract keypoints for good matches
        points1 = np.float32([keypoints1[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
        points2 = np.float32([keypoints2[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)

        # Find homography using RANSAC
        M, mask = cv2.findHomography(points2, points1, cv2.RANSAC, 5.0)

        # Warp image2 to image1 using homography
        registered_img = cv2.warpPerspective(img2, M, (img1.shape[1], img1.shape[0]))

        # Convert registered image to grayscale
        registered_img_gray = cv2.cvtColor(registered_img, cv2.COLOR_BGR2GRAY)

        # Apply edge detection using Prewitt
        kernel_x = np.array([[1, 0, -1], [1, 0, -1], [1, 0, -1]])
        kernel_y = np.array([[1, 1, 1], [0, 0, 0], [-1, -1, -1]])
        sobel_x = cv2.filter2D(registered_img_gray, -1, kernel_x)
        sobel_y = cv2.filter2D(registered_img_gray, -1, kernel_y)
        edges_registered = np.sqrt(sobel_x**2 + sobel_y**2)

        # Convert edges_registered to the same data type as gt_image
        edges_registered = edges_registered.astype(gt_image.dtype)

        # Compute SSIM and MSE
        ssim_index = ssim(edges_registered, gt_image)
        mse_value = mse(edges_registered, gt_image)

        return ssim_index, mse_value

    else:
        return None, None

base_dir = '/content/drive/MyDrive/data/train'
folder_a_path = os.path.join(base_dir, 'A')
folder_b_path = os.path.join(base_dir, 'B')
label_path = os.path.join(base_dir, 'label')

folder_a_images = os.listdir(folder_a_path)
folder_b_images = os.listdir(folder_b_path)
label_images = os.listdir(label_path)

ssim_scores = []
mse_scores = []

# Iterate through each image pair
for filename_a, filename_b, filename_label in zip(folder_a_images, folder_b_images, label_images):
    ssim_score, mse_score = process_image_pair(filename_a, filename_b, filename_label)
    if ssim_score is not None:
        ssim_scores.append(ssim_score)
        mse_scores.append(mse_score)
    else:
        print("Not enough matches for an image pair.")

mean_ssim = np.mean(ssim_scores)
mean_mse = np.mean(mse_scores)

print(f"Mean SSIM score: {mean_ssim}")
print(f"Mean MSE score: {mean_mse}")


Not enough matches for an image pair.
Not enough matches for an image pair.
Not enough matches for an image pair.
Not enough matches for an image pair.
Not enough matches for an image pair.
Not enough matches for an image pair.
Not enough matches for an image pair.
Not enough matches for an image pair.
Mean SSIM score: 0.4993893572244242
Mean MSE score: 2937.5731402207293


In [ ]:
import os
import cv2
import numpy as np
from skimage import io
from skimage.metrics import structural_similarity as ssim, mean_squared_error as mse

# Load image with OpenCV
def load_image_cv(path):
    return cv2.imread(path, cv2.IMREAD_COLOR)

def process_image_pair(filename_a, filename_b, filename_label):
    img1_path = os.path.join(folder_a_path, filename_a)
    img1 = load_image_cv(img1_path)

    img2_path = os.path.join(folder_b_path, filename_b)
    img2 = load_image_cv(img2_path)

    gt_image_path = os.path.join(label_path, filename_label)
    gt_image = io.imread(gt_image_path)

    img1_gray = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    img2_gray = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

    # Create SIFT detector
    sift = cv2.SIFT_create()

    # Detect keypoints and compute descriptors
    keypoints1, descriptors1 = sift.detectAndCompute(img1_gray, None)
    keypoints2, descriptors2 = sift.detectAndCompute(img2_gray, None)

    # FLANN parameters
    FLANN_INDEX_KDTREE = 1
    index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=3)
    search_params = dict(checks=70)
    flann = cv2.FlannBasedMatcher(index_params, search_params)

    # Match descriptors using FLANN
    matches = flann.knnMatch(descriptors1, descriptors2, k=2)

    # Ratio test to find good matches
    good_matches = []
    for m, n in matches:
        if m.distance < 0.75 * n.distance:
            good_matches.append(m)

    if len(good_matches) > 10:
        # Extract keypoints for good matches
        points1 = np.float32([keypoints1[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
        points2 = np.float32([keypoints2[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)

        # Find homography using RANSAC
        M, mask = cv2.findHomography(points2, points1, cv2.RANSAC, 5.0)

        # Warp image2 to image1 using homography
        registered_img = cv2.warpPerspective(img2, M, (img1.shape[1], img1.shape[0]))

        # Convert registered image to grayscale
        registered_img_gray = cv2.cvtColor(registered_img, cv2.COLOR_BGR2GRAY)

        # Apply Gaussian blur
        registered_img_blur = cv2.GaussianBlur(registered_img_gray, (3, 3), 0)

        # Apply Laplacian of Gaussian (LOG)
        edges_registered = cv2.Laplacian(registered_img_blur, cv2.CV_64F)

        # Convert edges_registered to the same data type as gt_image
        edges_registered = edges_registered.astype(gt_image.dtype)

        # Compute SSIM and MSE
        ssim_index = ssim(edges_registered, gt_image)
        mse_value = mse(edges_registered, gt_image)

        return ssim_index, mse_value

    else:
        return None, None

base_dir = '/content/drive/MyDrive/data/train'
folder_a_path = os.path.join(base_dir, 'A')
folder_b_path = os.path.join(base_dir, 'B')
label_path = os.path.join(base_dir, 'label')

folder_a_images = os.listdir(folder_a_path)
folder_b_images = os.listdir(folder_b_path)
label_images = os.listdir(label_path)

ssim_scores = []
mse_scores = []

# Iterate through each image pair
for filename_a, filename_b, filename_label in zip(folder_a_images, folder_b_images, label_images):
    ssim_score, mse_score = process_image_pair(filename_a, filename_b, filename_label)
    if ssim_score is not None:
        ssim_scores.append(ssim_score)
        mse_scores.append(mse_score)
    else:
        print("Not enough matches for an image pair.")

mean_ssim = np.mean(ssim_scores)
mean_mse = np.mean(mse_scores)

print(f"Mean SSIM score: {mean_ssim}")
print(f"Mean MSE score: {mean_mse}")


Not enough matches for an image pair.
Not enough matches for an image pair.
Not enough matches for an image pair.
Not enough matches for an image pair.
Not enough matches for an image pair.
Not enough matches for an image pair.
Not enough matches for an image pair.
Mean SSIM score: 0.29569466300430763
Mean MSE score: 18359.52167043294


In [ ]:
import os
import cv2
import numpy as np
from skimage import io
from skimage.metrics import structural_similarity as ssim, mean_squared_error as mse

# Load image with OpenCV
def load_image_cv(path):
    return cv2.imread(path, cv2.IMREAD_COLOR)

def process_image_pair(filename_a, filename_b, filename_label):
    img1_path = os.path.join(folder_a_path, filename_a)
    img1 = load_image_cv(img1_path)

    img2_path = os.path.join(folder_b_path, filename_b)
    img2 = load_image_cv(img2_path)

    gt_image_path = os.path.join(label_path, filename_label)
    gt_image = io.imread(gt_image_path)

    img1_gray = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    img2_gray = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

    # Create SIFT detector
    sift = cv2.SIFT_create()

    # Detect keypoints and compute descriptors
    keypoints1, descriptors1 = sift.detectAndCompute(img1_gray, None)
    keypoints2, descriptors2 = sift.detectAndCompute(img2_gray, None)

    # FLANN parameters
    FLANN_INDEX_KDTREE = 1
    index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=10)
    search_params = dict(checks=100)
    flann = cv2.FlannBasedMatcher(index_params, search_params)

    # Match descriptors using FLANN
    matches = flann.knnMatch(descriptors1, descriptors2, k=2)

    # Ratio test to find good matches
    good_matches = []
    for m, n in matches:
        if m.distance < 0.90 * n.distance:
            good_matches.append(m)

    if len(good_matches) > 10:
        # Extract keypoints for good matches
        points1 = np.float32([keypoints1[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
        points2 = np.float32([keypoints2[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)

        # Find homography using RANSAC
        M, mask = cv2.findHomography(points2, points1, cv2.RANSAC, 5.0)

        # Warp image2 to image1 using homography
        registered_img = cv2.warpPerspective(img2, M, (img1.shape[1], img1.shape[0]))

        # Convert registered image to grayscale
        registered_img_gray = cv2.cvtColor(registered_img, cv2.COLOR_BGR2GRAY)

        # Apply Scharr edge detection
        edges_registered = cv2.Scharr(registered_img_gray, -1, 1, 0)

        # Convert edges_registered to the same data type as gt_image
        edges_registered = edges_registered.astype(gt_image.dtype)

        # Compute SSIM and MSE
        ssim_index = ssim(edges_registered, gt_image)
        mse_value = mse(edges_registered, gt_image)

        return ssim_index, mse_value

    else:
        return None, None

base_dir = '/content/drive/MyDrive/data/train'
folder_a_path = os.path.join(base_dir, 'A')
folder_b_path = os.path.join(base_dir, 'B')
label_path = os.path.join(base_dir, 'label')

folder_a_images = os.listdir(folder_a_path)
folder_b_images = os.listdir(folder_b_path)
label_images = os.listdir(label_path)

ssim_scores = []
mse_scores = []

# Iterate through each image pair
for filename_a, filename_b, filename_label in zip(folder_a_images, folder_b_images, label_images):
    ssim_score, mse_score = process_image_pair(filename_a, filename_b, filename_label)
    if ssim_score is not None:
        ssim_scores.append(ssim_score)
        mse_scores.append(mse_score)
    else:
        print("Not enough matches for an image pair.")

mean_ssim = np.mean(ssim_scores)
mean_mse = np.mean(mse_scores)

print(f"Mean SSIM score: {mean_ssim}")
print(f"Mean MSE score: {mean_mse}")


Mean SSIM score: 0.41610727504900197
Mean MSE score: 6733.718012053243
